In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import spacy
nlp = spacy.load("en_core_web_sm")
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl


[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [3]:
import re
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:
df = pd.read_excel("WhatsApp_Messages.xlsx")

In [5]:
df.describe()

,DateTime,Sender,Message
count,1936,1936,1936
unique,987,3,1561
top,"30/06/2024, 11:51 am",Aditya Shinde,<Media omitted>\n
freq,11,782,87


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936 entries, 0 to 1935
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  1936 non-null   object
 1   Sender    1936 non-null   object
 2   Message   1936 non-null   object
dtypes: object(3)
memory usage: 45.5+ KB


In [7]:
df

,DateTime,Sender,Message
0,"23/05/2023, 8:00 pm",Aditya Shinde,@919604177822 kadhi free ahes\n
1,"23/05/2023, 8:01 pm",Om Sonawale,26\n
2,"23/05/2023, 8:03 pm",Sumedh Choudhari,Zhavnya Teva pan kahi tari karan kadhshil\n
3,"23/05/2023, 8:03 pm",Om Sonawale,Hehe🫡\n
4,"23/05/2023, 8:04 pm",Om Sonawale,U guys know me better\n
...,...,...,...
1931,"05/07/2024, 8:33 pm",Aditya Shinde,Udya ws la asel re mi\n
1932,"05/07/2024, 8:34 pm",Om Sonawale,🧢\n
1933,"05/07/2024, 8:34 pm",Aditya Shinde,Kharach challoe\n
1934,"05/07/2024, 9:21 pm",Sumedh Choudhari,Sunday?\n


cleaning the text

In [8]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]','',text)
    return text

In [9]:
df['Message'] = df['Message'].apply(clean_text)

tokenizing text

In [10]:
tokenizer =Tokenizer()
tokenizer.fit_on_texts(df['Message'])
sequence = tokenizer.texts_to_sequences(df['Message'])

creating sequences for training

In [11]:
max_len = 10
padded_sequences = pad_sequences(sequence, maxlen=max_len, padding='pre')

In [12]:
input_sequences = padded_sequences[:, :-1]
output_sequences = padded_sequences[:, -1]
output_sequences = keras.utils.to_categorical(output_sequences, num_classes=len(tokenizer.word_index)+1)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

#Defining the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,output_dim=100, input_length=max_len-1 ))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(input_sequences, output_sequences, epochs=100, batch_size=64)

Epoch 1/100


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.0740 - loss: 7.1817
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0834 - loss: 5.8886
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0842 - loss: 5.7688
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0846 - loss: 5.6064
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.0861 - loss: 5.5644
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.0988 - loss: 5.4352
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.1184 - loss: 5.3416
Epoch 8/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1222 - loss: 5.2613
Epoch 9/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.1190 - loss: 5.1611
Epoch 10/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.1233 - loss: 5.0161
Epoch 11/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1160 - loss: 4.9451
Epoch 12/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy:

In [23]:
# function to predict the nxt word
def predict_next_word(model, tokenizer, text, max_len):
    text = clean_text(text)
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len-1, padding='pre')
    predicted_prob = model.predict(padded_sequence, verbose=0)
    predicted_word_index = predicted_prob.argmax(axis=-1)[0]
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
        
text = "udya"
next_word = predict_next_word(model, tokenizer, text, max_len)
print(f"Next word: {next_word}")

Next word: na
